In [2]:
import pandas as pd
import numpy as np

# Custom date parser function
def parse_date(date, time):
    return pd.to_datetime(date + ' ' + time, format='%d/%m/%Y %H:%M:%S')

# Load the dataset
df = pd.read_csv('/Users/moji/PyTSF-MfG/data/household_power_consumption.csv', 
                 sep=';', 
                 parse_dates={'Timestamp': ['Date', 'Time']},
                 date_parser=parse_date,
                 dayfirst=True,
                 low_memory=False)
df

/var/folders/0s/whfbkk11083fmvg1wl2nrqqm0000gn/T/ipykernel_81876/2812190735.py:9: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv('/Users/moji/PyTSF-MfG/data/household_power_consumption.csv',
/var/folders/0s/whfbkk11083fmvg1wl2nrqqm0000gn/T/ipykernel_81876/2812190735.py:9: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv('/Users/moji/PyTSF-MfG/data/household_power_consumption.csv',


,Timestamp,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,2006-12-16 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,2006-12-16 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,2006-12-16 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,2006-12-16 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0
...,...,...,...,...,...,...,...,...
2075254,2010-11-26 20:58:00,0.946,0.000,240.430,4.000,0.000,0.000,0.0
2075255,2010-11-26 20:59:00,0.944,0.000,240.000,4.000,0.000,0.000,0.0
2075256,2010-11-26 21:00:00,0.938,0.000,239.820,3.800,0.000,0.000,0.0
2075257,2010-11-26 21:01:00,0.934,0.000,239.700,3.800,0.000,0.000,0.0


In [3]:
# Convert columns to appropriate data types
numeric_columns = ['Global_active_power', 'Global_reactive_power', 'Voltage', 
                   'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']

for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Set Timestamp as index
df.set_index('Timestamp', inplace=True)

# Check for missing values
print("Missing values before filling:")
print(df.isnull().sum())

Missing values before filling:
Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64


In [4]:
# Fill missing values
# For numeric columns, we'll use forward fill, then backward fill
df[numeric_columns] = df[numeric_columns].fillna(method='ffill').fillna(method='bfill')

print("\nMissing values after filling:")
print(df.isnull().sum())


Missing values after filling:
Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64


/var/folders/0s/whfbkk11083fmvg1wl2nrqqm0000gn/T/ipykernel_81876/1580426427.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[numeric_columns] = df[numeric_columns].fillna(method='ffill').fillna(method='bfill')


In [5]:
# Check for duplicate timestamps
duplicate_timestamps = df.index.duplicated()
print(f"\nNumber of duplicate timestamps: {duplicate_timestamps.sum()}")
# Remove duplicate timestamps (keep first occurrence)
# df = df[~duplicate_timestamps]


Number of duplicate timestamps: 0


In [7]:
# Resample to hourly data
df_hourly = df.resample('h').mean()

# Check for inconsistencies
print("\nDescriptive statistics for hourly data:")
print(df_hourly.describe())


Descriptive statistics for hourly data:
       Global_active_power  Global_reactive_power       Voltage  \
count         34589.000000           34589.000000  34589.000000   
mean              1.086200               0.123326    240.841505   
std               0.894942               0.067221      2.999127   
min               0.124000               0.000000    225.834500   
25%               0.341267               0.077667    239.197167   
50%               0.799533               0.106767    240.984500   
75%               1.574967               0.149367    242.691833   
max               6.560533               0.774333    251.902000   

       Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
count      34589.000000    34589.000000    34589.000000    34589.000000  
mean           4.604360        1.110951        1.287914        6.416515  
std            3.746337        3.518498        4.168720        7.344718  
min            0.503333        0.000000        0.000000    

In [8]:
# Check for outliers using IQR method
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Example: Check for outliers in 'Global_active_power'
outliers = detect_outliers(df_hourly, 'Global_active_power')
print(f"\nNumber of outliers in Global_active_power: {len(outliers)}")


Number of outliers in Global_active_power: 738


In [9]:
# Additional consistency checks
print("\nCheck for negative values in power and energy columns:")
for column in numeric_columns:
    negative_values = df_hourly[df_hourly[column] < 0]
    print(f"{column}: {len(negative_values)} negative values")


Check for negative values in power and energy columns:
Global_active_power: 0 negative values
Global_reactive_power: 0 negative values
Voltage: 0 negative values
Global_intensity: 0 negative values
Sub_metering_1: 0 negative values
Sub_metering_2: 0 negative values
Sub_metering_3: 0 negative values


In [10]:
# Save the cleaned and resampled data
df_hourly.to_csv('electricity_data.csv')

print("\nCleaned and resampled hourly data saved to 'electricity_data.csv'")


Cleaned and resampled hourly data saved to 'electricity_data.csv'
